<a href="https://colab.research.google.com/github/vaishjah3/ML-Algorithms/blob/main/Transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Data Augmentation

vgbgb